<a href="https://colab.research.google.com/github/RushiBShinde/ThinkForge-IE-643-project/blob/main/IE643_QA_on_llama_3_1_8b_4bit_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install Unsloth
!pip install "unsloth[colab-new]" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.7/276.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [ ]:
# 2. Load Model & Tokenizer (with explicit token fix)
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/llama-3.1-8b-Instruct"

# Load the model in 4-bit
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 32768,
    dtype = None,
    load_in_4bit = True,
)

# === CRITICAL FIX ===
# Llama 3.1 uses new special tokens. We must set them.
# <|eot_id|> is the "End of Turn" token.
tokenizer.add_special_tokens({
    "eos_token": "<|eot_id|>",
})
# Set the model's pad token to this new eos token
model.config.pad_token_id = tokenizer.eos_token_id
# ====================

# Move the model to the GPU
FastLanguageModel.for_inference(model)

print("--- Model and tokenizer loaded with correct Llama 3.1 tokens ---")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

--- Model and tokenizer loaded with correct Llama 3.1 tokens ---


In [ ]:
# 3. Define Your Task (Asking for a LIST of answers)

# --- Your Passage and Query (Unchanged) ---
passage = """The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 Englishto-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.8 after
training for 3.5 days on eight GPUs, a small fraction of the training costs of the
best models from the literature. We show that the Transformer generalizes well to
other tasks by applying it successfully to English constituency parsing both with
large and limited training data.
∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started
the effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer models and
has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head
attention and the parameter-free position representation and became the other person involved in nearly every
detail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and
tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and
efficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and
implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating
our research.
†Work performed while at Google Brain.
‡Work performed while at Google Research.
31st Conference on Neural Information Processing Systems (NIPS 2017), Long Beach, CA, USA.
arXiv:1706.03762v7 [cs.CL] 2 Aug 2023
1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
states ht, as a function of the previous hidden state ht−1 and the input for position t. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
significant improvements in computational efficiency through factorization tricks [21] and conditional
computation [32], while also improving model performance in case of the latter. The fundamental
constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences [2, 19]. In all but a few cases [27], however, such attention mechanisms
are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead
relying entirely on an attention mechanism to draw global dependencies between input and output.
The Transformer allows for significantly more parallelization and can reach a new state of the art in
translation quality after being trained for as little as twelve hours on eight P100 GPUs.
2 Background
The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU
[16], ByteNet [18] and ConvS2S [9], all of which use convolutional neural networks as basic building
block, computing hidden representations in parallel for all input and output positions. In these models,
the number of operations required to relate signals from two arbitrary input or output positions grows
in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes
it more difficult to learn dependencies between distant positions [12]. In the Transformer this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequencealigned recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [34].
To the best of our knowledge, however, the Transformer is the first transduction model relying
entirely on self-attention to compute representations of its input and output without using sequencealigned RNNs or convolution. In the following sections, we will describe the Transformer, motivate
self-attention and discuss its advantages over models such as [17, 18] and [9].
3 Model Architecture
Most competitive neural sequence transduction models have an encoder-decoder structure [5, 2, 35].
Here, the encoder maps an input sequence of symbol representations (x1, ..., xn) to a sequence
of continuous representations z = (z1, ..., zn). Given z, the decoder then generates an output
sequence (y1, ..., ym) of symbols one element at a time. At each step the model is auto-regressive
[10], consuming the previously generated symbols as additional input when generating the next.
2
Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder: The encoder is composed of a stack of N = 6 identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, positionwise fully connected feed-forward network. We employ a residual connection [11] around each of
the two sub-layers, followed by layer normalization [1]. That is, the output of each sub-layer is
LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer
itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding
layers, produce outputs of dimension dmodel = 512.
Decoder: The decoder is also composed of a stack of N = 6 identical layers. In addition to the two
sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head
attention over the output of the encoder stack. Similar to the encoder, we employ residual connections
around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions less than i.
3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vectors. The output is computed as a weighted sum
3
Scaled Dot-Product Attention Multi-Head Attention
Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several
attention layers running in parallel.
of the values, where the weight assigned to each value is computed by a compatibility function of the
query with the corresponding key.
3.2.1 Scaled Dot-Product Attention
We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of
queries and keys of dimension dk, and values of dimension dv. We compute the dot products of the
query with all keys, divide each by √
dk, and apply a softmax function to obtain the weights on the
values.
In practice, we compute the 'attention function on a set of queries simultaneously, packed together
into a matrix Q. The keys and values are also packed together into matrices K and V . We compute
the matrix of outputs as:
Attention(Q, K, V ) = softmax(QKT
√
dk
)V (1)
The two most commonly used attention functions are additive attention [2], and dot-product (multiplicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor
of √
1
dk
. Additive attention computes the compatibility function using a feed-forward network with
a single hidden layer. While the two are similar in theoretical complexity, dot-product attention is
much faster and more space-efficient in practice, since it can be implemented using highly optimized
matrix multiplication code.
While for small values of dk the two mechanisms perform similarly, additive attention outperforms
dot product attention without scaling for larger values of dk [3]. We suspect that for large values of
dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has
extremely small gradients 4
. To counteract this effect, we scale the dot products by √
1
dk
.
3.2.2 Multi-Head Attention
Instead of performing a single attention function with dmodel-dimensional keys, values and queries,
we found it beneficial to linearly project the queries, keys and values h times with different, learned
linear projections to dk, dk and dv dimensions, respectively. On each of these projected versions of
queries, keys and values we then perform the attention function in parallel, yielding dv-dimensional
4To illustrate why the dot products get large, assume that the components of q and k are independent random
variables with mean 0 and variance 1. Then their dot product, q · k =
Pdk
i=1 qiki, has mean 0 and variance dk.
4
output values. These are concatenated and once again projected, resulting in the final values, as
depicted in Figure 2.
Multi-head attention allows the model to jointly attend to information from different representation
subspaces at different positions. With a single attention head, averaging inhibits this.
MultiHead(Q, K, V ) = Concat(head1, ..., headh)WO
where headi = Attention(QWQ
i
, KW K
i
, V WV
i
)
Where the projections are parameter matrices W
Q
i ∈ R
dmodel×dk , W K
i ∈ R
dmodel×dk , WV
i ∈ R
dmodel×dv
and WO ∈ R
hdv×dmodel
.
In this work we employ h = 8 parallel attention layers, or heads. For each of these we use
dk = dv = dmodel/h = 64. Due to the reduced dimension of each head, the total computational cost
is similar to that of single-head attention with full dimensionality.
3.2.3 Applications of Attention in our Model
The Transformer uses multi-head attention in three different ways:
• In "encoder-decoder attention" layers, the queries come from the previous decoder layer,
and the memory keys and values come from the output of the encoder. This allows every
position in the decoder to attend over all positions in the input sequence. This mimics the
typical encoder-decoder attention mechanisms in sequence-to-sequence models such as
[38, 2, 9].
• The encoder contains self-attention layers. In a self-attention layer all of the keys, values
and queries come from the same place, in this case, the output of the previous layer in the
encoder. Each position in the encoder can attend to all positions in the previous layer of the
encoder.
• Similarly, self-attention layers in the decoder allow each position in the decoder to attend to
all positions in the decoder up to and including that position. We need to prevent leftward
information flow in the decoder to preserve the auto-regressive property. We implement this
inside of scaled dot-product attention by masking out (setting to −∞) all values in the input
of the softmax which correspond to illegal connections. See Figure 2.
3.3 Position-wise Feed-Forward Networks
In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully
connected feed-forward network, which is applied to each position separately and identically. This
consists of two linear transformations with a ReLU activation in between.
FFN(x) = max(0, xW1 + b1)W2 + b2 (2)
While the linear transformations are the same across different positions, they use different parameters
from layer to layer. Another way of describing this is as two convolutions with kernel size 1.
The dimensionality of input and output is dmodel = 512, and the inner-layer has dimensionality
df f = 2048.
3.4 Embeddings and Softmax
Similarly to other sequence transduction models, we use learned embeddings to convert the input
tokens and output tokens to vectors of dimension dmodel. We also use the usual learned linear transformation and softmax function to convert the decoder output to predicted next-token probabilities. In
our model, we share the same weight matrix between the two embedding layers and the pre-softmax
linear transformation, similar to [30]. In the embedding layers, we multiply those weights by √
dmodel.
5
Table 1: Maximum path lengths, per-layer complexity and minimum number of sequential operations
for different layer types. n is the sequence length, d is the representation dimension, k is the kernel
size of convolutions and r the size of the neighborhood in restricted self-attention."""

query = "What is attention function?"

# --- Check our work (Unchanged) ---
passage_tokens = tokenizer([passage], return_tensors="pt").input_ids.shape[1]
print(f"--- Document created with {passage_tokens} tokens ---")
# -------------------------


# === NEW SYSTEM PROMPT ===
# This prompt asks for a LIST of answers in a "results" array.
# It also provides a multi-item example to guide the model.
messages = [
    {
        "role": "system",
        "content": (
            "You are an expert analysis assistant. Your task is to analyze a query and a passage. "
            "First, find **all** exact sentences or phrases from the passage that **answer the query**. "
            "Second, for each extracted text, score how well it answers the query on a scale of 1-10. "
            "If no part of the passage answers the query, the 'results' list should be empty (`[]`).\n"
            "Provide your answer in the following JSON format:\n"
            "{\n"
            "  \"results\": [\n"
            "    {\n"
            "      \"relevance_score\": <score_1_to_10>,\n"
            "      \"extracted_text\": \"<first_extracted_text>\"\n"
            "    },\n"
            "    {\n"
            "      \"relevance_score\": <score_1_to_10>,\n"
            "      \"extracted_text\": \"<second_extracted_text>\"\n"
            "    }\n"
            "  ]\n"
            "}"
        )
    },
    {
        "role": "user",
        "content": f"Query: \"{query}\"\n\nPassage: \"{passage}\""
    }
]

# Use the tokenizer to apply the chat template
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Tokenize the final prompt
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

print(f"--- Total prompt size (document + prompt) is {inputs.input_ids.shape[1]} tokens ---")

--- Document created with 3050 tokens ---
--- Total prompt size (document + prompt) is 3258 tokens ---


In [ ]:
# 4. Run Inference (with more tokens)
import time

print("--- Starting inference (asking for multiple answers)... ---")
start_time = time.time()

outputs = model.generate(
    **inputs,
    # === CRITICAL CHANGE ===
    # We need to give the model *way* more space to write the list.
    max_new_tokens = 800,
    pad_token_id = tokenizer.eos_token_id,
    eos_token_id = tokenizer.eos_token_id,
    do_sample = False # Using deterministic greedy decoding
)

end_time = time.time()
print(f"--- Inference complete in {end_time - start_time:.2f} seconds ---")

--- Starting inference (asking for multiple answers)... ---
--- Inference complete in 43.66 seconds ---


In [ ]:
# 5. Decode and Print the Result

# We decode *only* the new tokens
text_output = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]

print("\n--- Model Output ---")
print(text_output)


--- Model Output ---
{
  "results": [
    {
      "relevance_score": 8,
      "extracted_text": "The attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key."
    },
    {
      "relevance_score": 9,
      "extracted_text": "We call our particular attention 'Scaled Dot-Product Attention' (Figure 2). The input consists of queries and keys of dimension dk, and values of dimension dv. We compute the dot products of the query with all keys, divide each by √dk, and apply a softmax function to obtain the weights on the values."
    },
    {
      "relevance_score": 8,
      "extracted_text": "Multi-head attention allows the model to jointly attend to information from different representation subspaces at different posi

# Preparing a Quantised version to load faster

In [ ]:
# 1. Install Unsloth
!pip install "unsloth[colab-new]" -q

In [ ]:
# 2. Load the model (This is the long wait)
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3.1-8b-Instruct",
    max_seq_length = 32768,
    dtype = None,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
save_directory = "/content/drive/My Drive/colab_models/llama-3.1-8b-4bit"

In [ ]:
# 5. Save the 4-bit model AND the tokenizer to Google Drive
# NOTE: This step will be SLOW. It has to copy ~5.5GB to your Drive.
print("--- Saving model to Google Drive... This will take a while! ---")
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"--- Model saved to {save_directory} ---")

--- Saving model to Google Drive... This will take a while! ---
--- Model saved to /content/drive/My Drive/colab_models/llama-3.1-8b-4bit ---


In [ ]:
# 1. Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # Force remount to be safe

Mounted at /content/drive


In [ ]:
# 2. Define paths
folder_path = "/content/drive/My Drive/colab_models/llama-3.1-8b-4bit"
parent_dir = "/content/drive/My Drive/colab_models/"
zip_file_name = "llama-3.1-8b-4bit.zip"
zip_file_path = f"{parent_dir}{zip_file_name}" # /content/drive/My Drive/colab_models/llama-3.1-8b-4bit.zip
folder_to_zip = "llama-3.1-8b-4bit" # The name of the folder itself

In [ ]:
# 3. First, delete the old, broken zip file
print(f"--- Removing any old/broken file at {zip_file_path} ---")
!rm -f "{zip_file_path}"

--- Removing any old/broken file at /content/drive/My Drive/colab_models/llama-3.1-8b-4bit.zip ---


In [ ]:
# 4. Create the zip file using the reliable `cd && zip` method
# This changes to the parent directory, then zips the folder.
# This will be slow (5-10+ mins) but you only do it ONCE.
print(f"--- Zipping {folder_path} ... This will take several minutes. ---")
!cd "{parent_dir}" && zip -r "{zip_file_name}" "{folder_to_zip}"

print(f"--- 🥳 Success! Your *real* zip file is now at {zip_file_path} ---")

--- Zipping /content/drive/My Drive/colab_models/llama-3.1-8b-4bit ... This will take several minutes. ---
  adding: llama-3.1-8b-4bit/ (stored 0%)
  adding: llama-3.1-8b-4bit/config.json (deflated 56%)
  adding: llama-3.1-8b-4bit/generation_config.json (deflated 37%)
  adding: llama-3.1-8b-4bit/model-00001-of-00002.safetensors

In [ ]:
# 1. Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # Force remount to be safe

# 2. Define paths
folder_path = "/content/drive/My Drive/colab_models/llama-3.1-8b-4bit"
parent_dir = "/content/drive/My Drive/colab_models/"
zip_file_name = "llama-3.1-8b-4bit.zip"
zip_file_path = f"{parent_dir}{zip_file_name}" # /content/drive/My Drive/colab_models/llama-3.1-8b-4bit.zip
folder_to_zip = "llama-3.1-8b-4bit" # The name of the folder itself

# 3. First, delete the old, broken zip file
print(f"--- Removing any old/broken file at {zip_file_path} ---")
!rm -f "{zip_file_path}"

# 4. Create the zip file using the reliable `cd && zip` method
# This changes to the parent directory, then zips the folder.
# This will be slow (5-10+ mins) but you only do it ONCE.
print(f"--- Zipping {folder_path} ... This will take several minutes. ---")
!cd "{parent_dir}" && zip -r "{zip_file_name}" "{folder_to_zip}"

print(f"--- 🥳 Success! Your *real* zip file is now at {zip_file_path} ---")

KeyboardInterrupt: 